## Deep Learning for classification multi-classe


1. Importation des bibliothèques Python + Préparation de l'environnement
2. Importation et prétraitement des données
3. Préparation de l'ensemble de données et du chargeur de données
4. Création du réseau de neurones pour le Fine Tunning
5. Ajustement du modèle
6. Validation de la performance du modèle
7. Sauvegarde du modèle et des artefacts pour l'avenir


Model de language utilisé : `DistilBERT`

* `DistilBERT`, c'est un modèle de transformer plus petit que `BERT` ou `RoBERTa`. Il est créé par un processus de distillation appliqué au modèle BERT :
    * [Blog-Post](https://medium.com/huggingface/distilbert-8cf3380435b5)
    * [Research Paper](https://arxiv.org/abs/1910.01108)
    * [Documentation for python](https://huggingface.co/transformers/model_doc/distilbert.html)



## Importation des bibliothèques Python + Préparation de l'environnement

In [ ]:
import pandas as pd # Pandas pour la gestion des données.

import torch #Library Pytorch classique nécéssaire pour les modèles de Deep Learning.

import transformers # Modèle Transformers sur lesquels reposent BERT+ variantess.

from torch.utils.data import Dataset, DataLoader # Gestion des données dans Pytorch

from transformers import DistilBertModel, DistilBertTokenizer # Modèle DisltilBERT et son tokenizer

### Initialisation de l'environement
1. Si un `GPU` est disponible alors nous faisons tourner nos scripts sur `GPU` sinon on utilise le `CPU`.
2. Définition de variables clés
3. Création de `Dataset`==> définit la façon dont le texte est prétraité avant d'être envoyé au réseau de neurones.
4. Définition de `Dataloader` ==> alimentera les données par lots au réseau pour un apprentissage et un traitement appropriés. 

Pour plus d'informations sur le Dataset et le Dataloader, consultez les [docs de PyTorch](https://pytorch.org/docs/stable/data.html)

In [ ]:
# GPU ou CPU ?

if torch.cuda.is_available():    
    # Dire à PyTorch d'utiliser le GPU.    
    device = torch.device("cuda")
    print('Il y a %d GPU(s) dispo.' % torch.cuda.device_count())
    print('Nous utiliserons le GPU:', torch.cuda.get_device_name(0))
else:
    print('Aucun GPU dispo, utilisation du CPU à la place.')
     # Dire à PyTorch d'utiliser le CPU. 
    device = torch.device("cpu")

In [ ]:
# Variables clés

MAX_LEN = 512 #Longueur maximale d'une description
TRAIN_BATCH_SIZE = 5 # Nombre d'éléments dans le batch d'apprentissage (combien d'éléments voir avant de calculer une iteration de descente de gradient)
VALID_BATCH_SIZE = 10 # Nombre d'éléments dans le batch de validation
EPOCHS = 1 # Nombre de passage du dataset complet
LEARNING_RATE = 1e-05 #t_k dans la descente de gradient


tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased') #Tokenizer de DistilBERT

## Classe `Triage` d'ensemble de données
- Cette classe est définie pour accepter la Dataframe comme entrée et générer des sorties symboliques qui sont utilisées par le modèle DistilBERT pour la formation. 
- Nous utilisons le tokenizer DistilBERT pour tokeniser les données dans la colonne "description" de la dataframe. 
- Le tokenizer utilise la méthode `encode_plus` pour effectuer la tokenisation et générer les sorties nécessaires, à savoir : ids`, `attention_mask` (masque d'attention)
- Pour en savoir plus sur le tokenizer, [se référer à ce document](https://huggingface.co/transformers/model_doc/distilbert.html#distilberttokenizer)
- La "cible" est la catégorie codée sur le titre de la nouvelle. 
- La classe *Triage* est utilisée pour créer 2 ensembles de données, pour la formation et pour la validation.
- L'ensemble de données pour la formation est utilisé pour affiner le modèle : **80 % des données d'origine**
- *L'ensemble de données de validation* est utilisé pour évaluer la performance du modèle. Le modèle n'a pas vu ces données pendant la formation. 

In [ ]:
class Triage(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len
        
    def __getitem__(self, index):
        desc = str(self.data.description[index])
        desc = " ".join(desc.split())
        inputs = self.tokenizer.encode_plus(
            desc,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True,
            truncation=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']

        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'targets': torch.tensor(self.data.Category[index], dtype=torch.long)
        } 
    
    def __len__(self):
        return self.len

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df_X = pd.read_json('./data/train.json').set_index('Id')
df_label=pd.read_csv('./data/train_label.csv').set_index('Id')

df=pd.concat([df_X, df_label], axis=1).drop(['gender'],axis=1)


# Creating the dataset and dataloader for the neural network

train_size = 0.85
train_dataset=df.sample(frac=train_size)
test_dataset=df.drop(train_dataset.index).reset_index(drop=True)
train_dataset = train_dataset.reset_index(drop=True)


print("FULL Dataset: {}".format(df.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(test_dataset.shape))

training_set = Triage(train_dataset, tokenizer, MAX_LEN)
testing_set = Triage(test_dataset, tokenizer, MAX_LEN)


#### Dataloader
- `Dataloader` est utilisé pour créer un dataloader d'apprentissage et de validation qui charge les données dans le réseau. Ceci est nécessaire car toutes les données de l'ensemble de données ne peuvent pas être chargées dans la mémoire en une seule fois, d'où la nécessité de contrôler la quantité de données chargées dans la mémoire et ensuite transmises au réseau.
- Ce contrôle est réalisé à l'aide de paramètres tels que `BATCH_SIZE` et `MAX_LEN`.
- Les `Dataloaders` de formation et de validation sont utilisés respectivement dans la partie formation et validation du flux.

In [ ]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

#### F1 Score

Pour évaluer notre modèle nous choisirons le f1 score.
Nous créeons une classe F1Score car il n'est pas implémenté en pytorch 

In [ ]:
from typing import Tuple

import torch


class F1Score:
    """
    Class for f1 calculation in Pytorch.
    """

    def __init__(self, average: str = 'macro'):
        """
        Init.

        Args:
            average: averaging method
        """
        self.average = average
        if average not in [None, 'micro', 'macro', 'weighted']:
            raise ValueError('Wrong value of average parameter')

    @staticmethod
    def calc_f1_micro(predictions: torch.Tensor, labels: torch.Tensor) -> torch.Tensor:
        """
        Calculate f1 micro.

        Args:
            predictions: tensor with predictions
            labels: tensor with original labels

        Returns:
            f1 score
        """
        true_positive = torch.eq(labels, predictions).sum().float()
        f1_score = torch.div(true_positive, len(labels))
        return f1_score

    @staticmethod
    def calc_f1_count_for_label(predictions: torch.Tensor,
                                labels: torch.Tensor, label_id: int) -> Tuple[torch.Tensor, torch.Tensor]:
        """
        Calculate f1 and true count for the label

        Args:
            predictions: tensor with predictions
            labels: tensor with original labels
            label_id: id of current label

        Returns:
            f1 score and true count for label
        """
        # label count
        true_count = torch.eq(labels, label_id).sum()

        # true positives: labels equal to prediction and to label_id
        true_positive = torch.logical_and(torch.eq(labels, predictions),
                                          torch.eq(labels, label_id)).sum().float()
        # precision for label
        precision = torch.div(true_positive, torch.eq(predictions, label_id).sum().float())
        # replace nan values with 0
        precision = torch.where(torch.isnan(precision),
                                torch.zeros_like(precision).type_as(true_positive),
                                precision)

        # recall for label
        recall = torch.div(true_positive, true_count)
        # f1
        f1 = 2 * precision * recall / (precision + recall)
        # replace nan values with 0
        f1 = torch.where(torch.isnan(f1), torch.zeros_like(f1).type_as(true_positive), f1)
        return f1, true_count

    def __call__(self, predictions: torch.Tensor, labels: torch.Tensor) -> torch.Tensor:
        """
        Calculate f1 score based on averaging method defined in init.

        Args:
            predictions: tensor with predictions
            labels: tensor with original labels

        Returns:
            f1 score
        """

        # simpler calculation for micro
        if self.average == 'micro':
            return self.calc_f1_micro(predictions, labels)

        f1_score = 0
        for label_id in range(1, len(labels.unique()) + 1):
            f1, true_count = self.calc_f1_count_for_label(predictions, labels, label_id)

            if self.average == 'weighted':
                f1_score += f1 * true_count
            elif self.average == 'macro':
                f1_score += f1

        if self.average == 'weighted':
            f1_score = torch.div(f1_score, len(labels))
        elif self.average == 'macro':
            f1_score = torch.div(f1_score, len(labels.unique()))

        return f1_score

## Création du réseau de neurones pour le Fine Tunning

Création d'un modèle personnalisé, en ajoutant le dropout (pour la régularisation) et une couche dense après le model `DistilBERT` pour obtenir le résultat final du modèle. 

Architecture :

Modèle BERT ==> Couche Dense ==> Drop out ==> Couche Dense 

In [ ]:
class DistillBERTClass(torch.nn.Module):
    def __init__(self):
        super(DistillBERTClass, self).__init__()
        # Définition des différentes couches du réseau
        
        # Couche du modèle DistilBERT
        self.l1 = DistilBertModel.from_pretrained("distilbert-base-uncased")
        
        # Couche dense
        self.pre_classifier = torch.nn.Linear(768, 768)
        
        # Module de Dropout
        # `0.3` est la probabilité qu'un neurone soit remis à zéro. 
        self.dropout = torch.nn.Dropout(0.4) 
        
        # Couche de classification
        # `28` est le nombre de classe à prédire. 
        self.classifier = torch.nn.Linear(768, 28)

    def forward(self, input_ids, attention_mask):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        pooler = self.pre_classifier(pooler)
        pooler = torch.nn.ReLU()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        return output

In [ ]:
torch.cuda.empty_cache()

In [ ]:
model = DistillBERTClass() #On instantie le modèle
model.to(device) # On "transfert" le modèle vers le device.

## Création de la fonction de perte et de l'optimiseur

In [ ]:
loss_function = torch.nn.CrossEntropyLoss() ## Fonction de perte CrossEntropyLoss : https://www.baeldung.com/cs/cross-entropy + https://en.wikipedia.org/wiki/Cross_entropy
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

## Fine Tuning du Modèle


Nous définissons ici une fonction `Train` qui entraîne le modèle sur l'ensemble de données d'apprentissage (`training_loader`), un nombre de fois spécifié (`EPOCH`) 

Les événements suivants se produisent dans cette fonction pour affiner le réseau de neurones :
- L'explorateur de données transmet les données au modèle en fonction de la taille du lot. 
- Les sorties ultérieures du modèle et la catégorie réelle sont comparées pour calculer la perte. 
- La valeur de la perte est utilisée pour optimiser le poids des neurones dans le réseau.
- La valeur de perte est imprimée dans la console tous les 5000 pas.

Comme vous pouvez le voir, à une époque, à la dernière étape, le modèle fonctionnait avec une perte minuscule de 0,0002485, c'est-à-dire que la sortie est extrêmement proche de la sortie réelle.

In [ ]:
# Fonction de calcul du F1_score
f1_metric=F1Score('macro')

In [ ]:
# Définition de la fonction d'apprentissage pour DistilBERT


def train(epoch):
    # Initialisation
    tr_loss = 0
    n_correct = 0
    n_faux = 0
    nb_tr_steps = 0
    nb_tr_examples = 0
    
    model.train() #On passe notre model en mode train en Pytorch
    
    for _,data in enumerate(training_loader, 0): # Pour chaque élement de notre jeu d'apprentissage 
        
        # On rend disponible pour le GPU les indices, masques et valeurs cibles 
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.long)
        
        # On passe nos données dans le modèle
        outputs = model(ids, mask)
        # On calcul la perte
        loss = loss_function(outputs, targets)
        tr_loss += loss.item()
        
        # On calcul l'accuracy
        big_val, big_idx = torch.max(outputs.data, dim=1)       
        MyF1_score=f1_metric(big_idx, targets)

        print(f'F1_score: {MyF1_score}')
        
        # On augmente notre compteur d'étapes
        nb_tr_steps += 1
        nb_tr_examples+=targets.size(0)
        
        #FF1=F1_Score(targets, big_idx,'macro')
        #F1_tr.append(FF1)
        
        # Tout les 5000 descritpions on affiche la perte et la précision
        if _%5000==0:
            loss_step = tr_loss/nb_tr_steps
            tp_step = (n_correct*100)/nb_tr_examples
              
            print(f"Training Loss per 5000 steps: {loss_step}")
            #print(f"Training Accuracy per 5000 steps: {accu_step}")
            #print(f"Training F1_score per 5000 steps: {FF1}")
            

        optimizer.zero_grad()
        loss.backward()
        # # When using GPU
        optimizer.step()

    print(f'The Total Accuracy for Epoch {epoch}: {(n_correct*100)/nb_tr_examples}')
    epoch_loss = tr_loss/nb_tr_steps
    epoch_accu = (n_correct*100)/nb_tr_examples
    print(f"Training Loss Epoch: {epoch_loss}")
    #print(f"Training Accuracy Epoch: {epoch_accu}")
    #print(f"Training F1_score Epoch: {FF1}")

    return 

In [ ]:
for epoch in range(EPOCHS):
    train(epoch)

## Validation du modèle  

In [ ]:
import numpy as np

arr=[]
def valid(model, testing_loader):
    model.eval()
    n_correct = 0; n_wrong = 0; total = 0
    
    tr_loss = 0
    nb_tr_steps = 0
    nb_tr_examples = 0
    with torch.no_grad():
        for _, data in enumerate(testing_loader, 0):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.long)
            outputs = model(ids, mask).squeeze()
            loss = loss_function(outputs, targets)
            tr_loss += loss.item()
            big_val, big_idx = torch.max(outputs.data, dim=1)
            
            #n_correct += calcuate_accu(big_idx, targets)
            MyF1_score=f1_metric(big_idx, targets)

            print(f'F1_score: {MyF1_score}')

            arr.append(MyF1_score)

            nb_tr_steps += 1
            print(nb_tr_steps)
            nb_tr_examples+=targets.size(0)
 
    #FF1=f1_score(targets, big_idx, average='macro')
    #print(f"Training F1_score Epoch: {FF1}")
    return MyF1_score


In [ ]:
print('This is the validation section to print the accuracy and see how it performs')
print('Here we are leveraging on the dataloader crearted for the validation dataset, the approcah is using more of pytorch')

acc = valid(model, testing_loader)
print("Accuracy on test data = %0.2f%%" % acc)

In [ ]:
for item in f1arr:
  item.compute()
  

In [ ]:
print(numpy.array(f1arr).mean())

## Sauvegarde

In [ ]:
# Saving the files for re-use

output_model_file = '/content/drive/My Drive/models/pytorch_distilbert_news.bin'
output_vocab_file = '/content/drive/My Drive/models/vocab_distilbert_news.bin'

model_to_save = model
torch.save(model_to_save, output_model_file)
tokenizer.save_vocabulary(output_vocab_file)

print('All files saved')